In [2]:
import pandas as pd
import numpy as np
import geocoder

In [3]:
sensor = pd.read_csv('ParkingSensorData.csv')

/Users/Grace/anaconda/envs/fastai/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
sensor.drop(sensor.tail(1).index, inplace=True)

In [11]:
sensor.head()

,BLOCK_ID,STREET_NAME,BLOCK_NUM,STREET_BLOCK,AREA_TYPE,PM_DISTRICT_NAME,RATE,START_TIME_DT,TOTAL_TIME,TOTAL_OCCUPIED_TIME,...,GMP_UNKNOWN_TIME,COMM_TIME,COMM_OCCUPIED_TIME,COMM_VACANT_TIME,COMM_UNKNOWN_TIME,CAL_MONTH_NAME,CAL_YEAR,CAL_DATE,DAY_TYPE,TIME_OF_DAY
0,41522,FILLMORE ST,22.0,FILLMORE ST 2200,Pilot,Fillmore,NaN,01-jul-2012 21:00:00,61200.0,35847.0,...,0.0,0.0,0.0,0.0,0.0,July,2012.0,01-jul-2012,weekend,2100.0
1,41524,FILLMORE ST,24.0,FILLMORE ST 2400,Pilot,Fillmore,NaN,01-jul-2012 22:00:00,64800.0,27576.0,...,0.0,0.0,0.0,0.0,0.0,July,2012.0,01-jul-2012,weekend,2200.0
2,50001,JACKSON ST,1.0,JACKSON ST 100,Pilot,Downtown,NaN,01-jul-2012 02:00:00,79200.0,39985.0,...,0.0,0.0,0.0,0.0,0.0,July,2012.0,01-jul-2012,weekend,200.0
3,50002,JACKSON ST,2.0,JACKSON ST 200,Pilot,Downtown,NaN,01-jul-2012 08:00:00,82800.0,30465.0,...,0.0,0.0,0.0,0.0,0.0,July,2012.0,01-jul-2012,weekend,800.0
4,10903,09TH AVE,3.0,09TH AVE 300,Control,Inner Richmond,NaN,01-jul-2012 08:00:00,93600.0,85690.0,...,0.0,0.0,0.0,0.0,0.0,July,2012.0,01-jul-2012,weekend,800.0


In [54]:
uniqueblock = list(sensor.STREET_BLOCK.unique())

In [14]:
def geocode(address):
    add = "%s, San Francisco, CA" %(address)
    cord = geocoder.google(add).latlng
    return cord

In [51]:
#set minibatch index list
n = len(uniqueblock)
l = np.arange(0,n,10)
if  n%10 != 0:
    l = np.append(l,n)

In [61]:
#geocode by small batch of 10
geo = []
for i in range(len(l)-1):
    blockgeo = [geocode(block) for block in uniqueblock[l[i]:l[i+1]]]
    geo = geo + blockgeo

In [68]:
#fill the Nones due to API
for i in range(len(geo)):
    if geo[i] == None:
        geo[i] = geocode(uniqueblock[i])

In [69]:
#Check None)
len([g for g in geo if g == None])

0

In [79]:
geoblock = pd.DataFrame({"Street_Block": uniqueblock, "Geocode": geo})[["Street_Block","Geocode"]]

In [80]:
geoblock.head()

,Street_Block,Geocode
0,FILLMORE ST 2200,"[37.7900066, -122.4339023]"
1,FILLMORE ST 2400,"[37.7917689, -122.4341879]"
2,JACKSON ST 100,"[37.7970989, -122.3983614]"
3,JACKSON ST 200,"[37.7967346, -122.3999102]"
4,09TH AVE 300,"[37.7827927, -122.4676431]"


In [81]:
geoblock['lat'] = geoblock['Geocode'].apply(lambda x:x[0])
geoblock['lng'] = geoblock['Geocode'].apply(lambda x:x[1])

In [82]:
geoblock

,Street_Block,Geocode,lat,lng
0,FILLMORE ST 2200,"[37.7900066, -122.4339023]",37.790007,-122.433902
1,FILLMORE ST 2400,"[37.7917689, -122.4341879]",37.791769,-122.434188
2,JACKSON ST 100,"[37.7970989, -122.3983614]",37.797099,-122.398361
3,JACKSON ST 200,"[37.7967346, -122.3999102]",37.796735,-122.399910
4,09TH AVE 300,"[37.7827927, -122.4676431]",37.782793,-122.467643
5,10TH AVE 300,"[37.7827304, -122.4687219]",37.782730,-122.468722
6,14TH AVE 2800,"[37.7379624, -122.4690183]",37.737962,-122.469018
7,BERRY ST 100,"[37.7734893, -122.3964525]",37.773489,-122.396452
8,BERRY ST 200,"[37.775109, -122.394747]",37.775109,-122.394747
9,FILLMORE ST 3100,"[37.7982378, -122.4355948]",37.798238,-122.435595


In [83]:
geoblock.to_csv("geoblock",index = False)